In [7]:
import pandas as pd
from  sqlalchemy import create_engine
import os

In [10]:
# Reemplaza con tus propios datos
user = 'root'
password = 'yaris'  # La contraseña que estableciste al instalar MySQL
host = '127.0.0.1'  # O 'localhost'
database = 'olist_db' # El nombre de la base de datos que creaste

# Creamos la cadena de conexión
connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}/{database}"

# Creamos el motor de conexión con SQLAlchemy
engine = create_engine(connection_string)

In [12]:
# Importamos las librerías necesarias
import pandas as pd
from sqlalchemy import create_engine
import os

# --- CONFIGURA TUS DATOS AQUÍ ---
user = 'root'
password = 'yaris'  # ¡Cambia esto por tu contraseña real!
host = '127.0.0.1'                  # O 'localhost'
database = 'olist_db'               # El nombre de tu base de datos

# --- CREAMOS EL MOTOR DE CONEXIÓN ---
connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}/{database}"
engine = create_engine(connection_string)

# Mensaje de confirmación (si no hay error, todo bien)
print("¡Motor de conexión a MySQL creado con éxito!")

¡Motor de conexión a MySQL creado con éxito!


In [16]:
# --- RUTA EXACTA A TU CARPETA DE DATOS ---
csv_folder_path = r'C:\Users\diego\Desktop\olist\olist_project\olist_dataset'

# --- BUCLE PARA CARGAR LOS ARCHIVOS ---
print(f"Buscando archivos CSV en: {csv_folder_path}\n")

for filename in os.listdir(csv_folder_path):
    if filename.endswith('.csv'):
        # Ruta completa a cada archivo CSV
        file_path = os.path.join(csv_folder_path, filename)
        
        # Leemos el archivo a un DataFrame
        df = pd.read_csv(file_path)
        
        # Creamos un nombre de tabla limpio
        table_name = filename.replace('olist_', '').replace('_dataset.csv', '').replace('.csv', '')
        
        print(f"Cargando '{filename}' en la tabla de MySQL '{table_name}'...")
        
        # Guardamos el DataFrame en MySQL usando nuestro 'engine'
        df.to_sql(table_name, con=engine, if_exists='replace', index=False, chunksize=10000)
        
print("\n¡Proceso de carga completado!")

Buscando archivos CSV en: C:\Users\diego\Desktop\olist\olist_project\olist_dataset

Cargando 'olist_customers_dataset.csv' en la tabla de MySQL 'customers'...
Cargando 'olist_geolocation_dataset.csv' en la tabla de MySQL 'geolocation'...
Cargando 'olist_orders_dataset.csv' en la tabla de MySQL 'orders'...
Cargando 'olist_order_items_dataset.csv' en la tabla de MySQL 'order_items'...
Cargando 'olist_order_payments_dataset.csv' en la tabla de MySQL 'order_payments'...
Cargando 'olist_order_reviews_dataset.csv' en la tabla de MySQL 'order_reviews'...
Cargando 'olist_products_dataset.csv' en la tabla de MySQL 'products'...
Cargando 'olist_sellers_dataset.csv' en la tabla de MySQL 'sellers'...
Cargando 'product_category_name_translation.csv' en la tabla de MySQL 'product_category_name_translation'...

¡Proceso de carga completado!


In [17]:
# Verificamos las tablas que se han creado en MySQL
query = "SHOW TABLES;"
tables_df = pd.read_sql(query, engine)

print("\n--- Tablas creadas en la base de datos 'olist_db' ---")
print(tables_df)


--- Tablas creadas en la base de datos 'olist_db' ---
                  Tables_in_olist_db
0                          customers
1                        geolocation
2                        order_items
3                     order_payments
4                      order_reviews
5                             orders
6  product_category_name_translation
7                           products
8                            sellers
